In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import requests
import geopandas as gpd
import pandas as pd

# Define broader Overpass API query: all artworks in Los Angeles
query = """
[out:json][timeout:60];
area["name"="Los Angeles"]->.searchArea;
(
  node["tourism"="artwork"](area.searchArea);
  way["tourism"="artwork"](area.searchArea);
  relation["tourism"="artwork"](area.searchArea);
);
out center;
"""

# Overpass API URL
url = "http://overpass-api.de/api/interpreter"

# Request data
response = requests.get(url, params={'data': query})

if response.status_code == 200:
    data = response.json()

    if 'remark' in data:
        print(f"Overpass API Error: {data['remark']}")
    else:
        elements = data.get("elements", [])
        locations = []

        for element in elements:
            tags = element.get("tags", {})
            lat = element.get("lat", element.get("center", {}).get("lat"))
            lon = element.get("lon", element.get("center", {}).get("lon"))

            if lat and lon:
                locations.append({
                    "id": element.get("id"),
                    "name": tags.get("name", "Unknown"),
                    "artwork_type": tags.get("artwork_type", ""),
                    "description": tags.get("description", ""),
                    "lat": lat,
                    "lon": lon
                })

        # Convert to DataFrame
        df = pd.DataFrame(locations)

        if df.empty:
            print("No artwork locations found.")
        else:
            # Combine fields for filtering
            df["combined_tags"] = (
                df["name"].fillna('') + " " +
                df["artwork_type"].fillna('') + " " +
                df["description"].fillna('')
            )

            # Filter for relevant artwork types
            keywords = "mural|graffiti|stencil|streetart|street_art|street art"
            filtered_df = df[df["combined_tags"].str.contains(keywords, case=False, na=False)].copy()

            if filtered_df.empty:
                print("No murals or related artwork types found after filtering.")
            else:
                # Convert to GeoDataFrame
                gdf = gpd.GeoDataFrame(
                    filtered_df,
                    geometry=gpd.points_from_xy(filtered_df["lon"], filtered_df["lat"]),
                    crs="EPSG:4326"
                )

                # Save to CSV
                gdf.to_csv("la_street_art_filtered.csv", index=False)
                print(f"Filtered dataset saved as CSV: la_street_art_filtered.csv")
                print(f"Number of filtered results: {len(gdf)}")
                print(gdf[["name", "artwork_type", "lat", "lon"]].head())

else:
    print(f"Error fetching data from Overpass API. Status code: {response.status_code}")
    print(f"Response content: {response.text}")


Filtered dataset saved as CSV: la_street_art_filtered.csv
Number of filtered results: 84
                                name artwork_type        lat         lon
23         Smile South Central Mural        mural  33.993036 -118.320270
28           LA River - Valley Biome        mural  34.186937 -118.604060
29     LA River - Connect the Valley        mural  34.187078 -118.603778
36                           Unknown        mural  34.211964 -118.448931
37  Elliot Smith "Figure 8" Memorial        mural  34.095182 -118.283681


In [4]:
gdf.to_csv("/content/drive/My Drive/Colab Notebooks/la_street_art_filtered.csv", index=False)
